<a href="https://colab.research.google.com/github/congthanh-tk/Sentiment-Analysis/blob/main/CNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdown

In [6]:
import gdown

url = 'https://drive.google.com/uc?id=19XUp8VXAth078WwnBFVVN5u-DrJehrAb'
output = 'data.rar'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19XUp8VXAth078WwnBFVVN5u-DrJehrAb
To: /content/data.rar
100%|██████████| 2.36G/2.36G [00:23<00:00, 102MB/s] 


'data.rar'

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

n_node_max = 224
n_class = 2
n_edge_type = 13
np.random.seed(10)

In [8]:
def get_img(edge_data):
  n_node = edge_data.shape[0] + 1
  img = np.zeros((n_node, n_node, n_edge_type))
  data = edge_data.drop(["start", "end"], axis = 1).to_numpy()
  for index, row in edge_data[["start", "end"]].astype(np.int32).iterrows():
    start, end = row["start"], row["end"]
    img[start][end] = data[index]
  return img

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling1D,Input,concatenate,Concatenate


def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(n_node_max,n_node_max,n_edge_type)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_class, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [23]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

def get_model(input_size):
    # Su dung VGG16
    model_vgg16_conv = VGG16(
        input_shape=(n_node_max, n_node_max, n_edge_type),
        classes=n_class,
        weights=None,
        classifier_activation="softmax", 
        include_top=False)

    # Dong bang cac layer
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    # Tao model
    input = Input(shape=input_size, name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)

    # Them cac layer FC va Dropout
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_class, activation='softmax', name='predictions')(x)

    # Compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.FalseNegatives()])

    return my_model

In [ ]:
!unzip "/content/data.rar"

In [ ]:
import os
import gc
def clear_error_data(input_path) :
  for root, dirs, files in os.walk(input_path, topdown=False):
    for dir in dirs:
        if ".c" in dir :
          path = os.path.join(root, dir, "edges.csv")
          print(path)
          try:
            edge_data = pd.read_csv(path)
            img = get_img(edge_data)
          except Exception as e:
            print("+++++++++++++++++++++")
            print(e)
            os.remove(path)
          gc.collect()

In [12]:
import os
def load_dataset_by_path(input_path) :
  data = []
  labels = []
  for root, dirs, files in os.walk(input_path, topdown=False):
    for dir in dirs:
        if ".c" in dir:
          edge_path = os.path.join(root, dir, "edges.csv")
          if os.path.exists(edge_path):
            labels.append(1 if "_1.c" in dir else 0)
            data.append(edge_path)
  return data, labels

In [13]:
# Load data
# clear_error_data("/content/data/")
data, label = load_dataset_by_path("/content/data/")

In [14]:
# Fit to model
from tensorflow.keras.utils import to_categorical
label_binary = to_categorical(label)
X_train, X_test, y_train, y_test = train_test_split(data, label_binary, test_size=0.3)

In [15]:
d = {'data': X_train, 'label': y_train.argmax(axis=1)}
df = pd.DataFrame(data=d)
df

,data,label
0,/content/data/processed/1646_1.c/edges.csv,1
1,/content/data/processed/10082_0.c/edges.csv,0
2,/content/data/processed/2627_0.c/edges.csv,0
3,/content/data/processed/2121_0.c/edges.csv,0
4,/content/data/processed/3366_0.c/edges.csv,0
...,...,...
12703,/content/data/processed/3129_0.c/edges.csv,0
12704,/content/data/processed/703_1.c/edges.csv,1
12705,/content/data/processed/9922_0.c/edges.csv,0
12706,/content/data/processed/15092_0.c/edges.csv,0


In [16]:
# Dung de tao toan bo du lieu va load theo batch
class Dataset:
    def __init__(self, data, label, w, h):
        # the paths of images
        self.data = np.array(data) # the paths of segmentation images
        # Label
        self.label = np.array(label)
        # Shape
        self.w = w
        self.h = h

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # read data
        edge_data = pd.read_csv(self.data[i])
        image = get_img(edge_data)
        image = cv2.resize(image, (self.w, self.w))
        label = self.label[i]
        return image, label

In [17]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size, size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size = size

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)

    def __len__(self):
        return self.size // self.batch_size

In [18]:
# Build dataaset
train_dataset = Dataset(X_train, y_train, n_node_max, n_edge_type)
test_dataset = Dataset(X_test, y_test, n_node_max, n_edge_type)

In [19]:
# Loader
train_loader = Dataloader(train_dataset, 16, len(train_dataset))
test_loader = Dataloader(test_dataset, 16, len(test_dataset))

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/checkpoint_9_10.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',save_best_only=True)
# model = create_cnn_model()
model = tf.keras.applications.VGG16(
    input_shape=(n_node_max, n_node_max, n_edge_type),
    classes=n_class,
    weights=None,
    classifier_activation="softmax",
)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
# model = get_model(input_size=(n_node_max, n_node_max, n_edge_type))
# history = model.fit( x = X_train, y = Y_train, batch_size=50, epochs=50,callbacks = [model_checkpoint_callback], validation_split = 0.2)
history = model.fit(train_loader, validation_data=test_loader, epochs=50, callbacks = [model_checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/keras/applications/vgg16.py:134: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 13 input channels.
  weights=weights)


Epoch 1/50
494/794 [=================>............] - ETA: 11:28 - loss: 0.4532 - binary_accuracy: 0.9100 - false_negatives_1: 711.0000

In [ ]:
model.save('/content/drive/MyDrive/InformationSecurity/checkpoint_7_12.hdf5')

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/InformationSecurity/checkpoint_9_10.hdf5'
model = create_cnn_model()
model.load_weights(checkpoint_filepath)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      3776      
                                                                 
 conv2d_1 (Conv2D)           (None, 254, 254, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 127, 127, 32)      9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 32)      9248      
                                                        

In [ ]:
from sklearn.metrics import  *

y_pred = model.predict_generator(test_loader)
y_true = y_test.argmax(axis = 1)
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ignored

In [ ]:
import collections
collections.Counter(y_pred.argmax(axis = 1))

Counter({0: 5440})

In [ ]:
y_true = y_test.argmax(axis = 1)[:5440]
print(classification_report(y_true, y_pred.argmax(axis = 1)))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4939
           1       0.00      0.00      0.00       501

    accuracy                           0.91      5440
   macro avg       0.45      0.50      0.48      5440
weighted avg       0.82      0.91      0.86      5440



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(matrix, annot=True, fmt=".0f", cmap='Blues', yticklabels=['non-vul', 'vul'],xticklabels=['non-vul', 'vul'])
# plt.xlabel("Predicted")
# plt.ylabel("Actual") 